# Limpieza de los datos

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from collections import Counter
import re
import nltk
nltk.download(['punkt','stopwords','wordnet','words'])
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#import unidecode

[nltk_data] Downloading package punkt to /home/cmejia3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cmejia3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/cmejia3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/cmejia3/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
from nltk import *
from nltk.corpus import *

In [3]:
base_path =  Path("~").expanduser().resolve()
#base_path = Path.cwd().expanduser().resolve()
input_file_path  = base_path / 'datasets/papers-txt/'
#datasetOut =base_path / "datasets/salidas_procesamiento/"
#datasetOut_freq = base_path / "datasets/salidas_freq/"

In [4]:
input_file_path

PosixPath('/home/cmejia3/datasets/papers-txt')

# Preprocesamiento

In [5]:
def fileSize(fileIn):
    size=os.stat(fileIn).st_size
    return size

def wordsCount(contenido):
    totWwords=contenido.split()
    return len(totWwords)

def cleanWordCount(contenido):
    contenido =re.sub('(f|ht)tp(s?)://(.*)[.][a-z]+',' ',contenido) # Eliminar las URL
    #contenido =re.sub('REFERENCES (\S|\w)+',' ',contenido) # Eliminar la bibliografia
    contenido =re.sub('[a-zA-Z0-9.?{}]+@\w+\.\w+.\w*','',contenido) # Eliminar los correos
    contenido =re.sub('\[[a-zA-Z0-9\,\. ]+\]','',contenido) # Eliminar cualquier contenido entre corchetes
    contenido =re.sub('\([a-zA-Z0-9\,\.\- ]+\)',' ',contenido) # Eliminar cualquier contenido entre paréntesis
    contenido =re.sub('((et al\.)|(i\.i\.d\.)|(i\.e\.)|\-|\'|\’|\`)','',contenido) # Eliminar abreviaciones, apostrofes y guion
    #contenido =re.sub('(f|F)igure [0-9]+.[0-9]',' ',contenido) # Eliminar Figure
    contenido =re.sub('[^a-zA-Z_á\éíóúà\èìòùäëïöü\s]','',contenido) # Eliminar caracteres que no sean: letra, número o vocales acentuadas
    contenido =re.sub(' +',' ',contenido) # Eliminar espacios en blanco
    contenido =re.sub('(a-z|A-Z){1,1}','',contenido) # Eliminar palabras o números de un caracter de longitud   
    #contenido =re.sub('[^A-Za-z0-9.,_%+-\(\)\[\]\´\'\`]',' ',contenido)
    #contenido =re.sub('\[(0-9)+\]',' ',contenido)    
    #totWordDepurado = Counter(map(str, contenido.split()))
    #outputFile= open(datasetOut, 'w', encoding='UTF-8')
    #outputFile.write(contenido)
    #outputFile.close()
    totWwords=contenido.split()
    setWords = set(totWwords)
    #print("Total de palabras {}".format(len(totWwords)))
    #print("Total de palabras después del pre-procesamiento: {}".format(totWordDepurado))
    return len(totWwords),contenido,setWords

# Procesamiento

In [6]:
def lang_ratio(input):
    lang_ratio = {}
    tokens = wordpunct_tokenize(input)
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        word_set = set(words)
        common_elements = word_set.intersection(stopwords_set)
        lang_ratio[language] = len(common_elements)
    return lang_ratio

def detect_language(input):
    ratios = lang_ratio(input)
    language = max(ratios, key = ratios.get)
    return language

Conjunto de las stop words que serán eliminadas ya que no aportan valor:

In [7]:
stopWords = stopwords.words('english')

Palabras que considermos StopWords que no estan incluidas en el conjunto descargado:

In [8]:
listoStopWords = ['www','https','html','figure', 'chapter','abbcbccabcabcabcabcbcbabacba']

Se añaden las palabras que consideramos al conjunto principal

In [9]:
stopWords.extend(listoStopWords)

In [10]:
dictonary = nltk.corpus.words.words()

### Función principal procesamiento

In [11]:
def clean_files(texto,stopWords):
    
    #Quitar todos los acentos
    #texto = unidecode.unidecode(texto)
    
    #Quitar todos los caracteres especiales
    texto = re.sub('[^A-Za-z0-9]+',' ',texto)
    
    #Pasar todo a minisculas
    texto = texto.lower()
    
    #Tokenizar
    tokens = texto.split()
    
    #Variable que guarda el año en el que estamos que es el limite superior de los números que no se van a eliminar
    currentYear = int(dt.datetime.now().year)
    
    #Verificar que las palabras tengan más de un caracter, que además sean solo sean letras
    # o si son números que esten entre un rango que sea admisible para no eliminar información de año que se mencione en los artículos
    # y finalmente que no sean palabras que estan en el dicccionario de stopwords.
    
    #tokens = [w for w in tokens if (len(w)>1)&(w.isalpha() or (w.isnumeric() and int(w)>=1800 and int(w)<=currentYear))&(w not in stopWords)]
    tokens = [w for w in tokens if (len(w)>1)&(w.isalpha())&(w not in stopWords)]

    #Stemmer
    ps = PorterStemmer() 
    tokens = [ps.stem(w) for w in tokens]
    
    #Lematización
    word_net_lemmatizar = WordNetLemmatizer()

    tokens = [word_net_lemmatizar.lemmatize(w, pos = "v") for w in tokens]
    
    #Se retorna el texto nuevamente en un solo string luego de ser procesado
    to_return = ' '.join(tokens)
    
    #Se retorna el vocabulario de cada documento
    set_words = set(tokens)
    
    #Y la frecuencia de las palabras
    freq = nltk.FreqDist(tokens)
    return to_return,set_words,freq

Inicialización de los conjuntos:

    - Vocabulary: el conjunto de todas las palabras que contienen los documentos
    - results_text: la lista con los documentos ya organizados para construir el bag of words
    - results_frecuency: información de cada documento de las palabras que contiene cuántas veces las contiene

In [12]:
vocabulary = set()
results_text = []
results_frecuency = []

## Limpieza de los datos

In [13]:
fileSummary = "CleanSummary.csv"

contenido= "Archivo" + ";" + "Tamaño(K)" + ";" + "Cant Palabras Inicial" + ";" + "Cant Palabras depuradas"+ ";" +"Porc Limpieza"+ "\n"
resultado = pd.DataFrame()
indexFiles = []
documents = []

for f in input_file_path.glob('*.txt'):
    #Peso del archivo
    tmpSize=round(fileSize(f)/1014)
    
    #Lectura del archivo
    input_file = open(f, "r", encoding = 'utf-8')
    input_aux = input_file.read()
    
    #Cuenta de palabras iniciales
    tmpWordsOri=wordsCount(input_aux)
    #out = datasetOut / str(f).split('/')[-1]
    
    #Cuenta de palabras despues de la limpieza
    tmpWordsEnd,text,setWord=cleanWordCount(input_aux)
    #tmpPerClean=round((tmpWordsEnd/tmpWordsOri)*100)
    
    
    #Detectando el idioma
    aux = detect_language(text)
    
    if(aux == 'english'):
        text_cleanned,set_words,freq = clean_files(text,stopWords)
        #out2 = datasetOut / str(f).split('/')[-1]
        documents.append(text_cleanned)
        vocabulary = vocabulary.union(set_words)
        results_text.append(text_cleanned)
        indexFiles.append(str(f).split('/')[-1])
        
        #Escritura de los resultados del preprocesamiento
        auxRes= pd.DataFrame({'Archivo': str(f).split('/')[-1], 'Tamaño(K)': [tmpSize], 'Cant Palabras Inicial': [tmpWordsOri],'Cant Palabras depuradas': [tmpWordsEnd],"Vocabulario Inicial":len(setWord),"Vocabulario Final":len(set_words)})
        resultado = pd.concat([resultado, auxRes])
        #break
    else:
        print(aux + ':' + str(f))

resultado.to_csv(fileSummary, sep = ';', index = False)

german:/home/cmejia3/datasets/papers-txt/1508.02340.txt


In [14]:
resultado.head()

,Archivo,Tamaño(K),Cant Palabras Inicial,Cant Palabras depuradas,Vocabulario Inicial,Vocabulario Final
0,1410.2670.txt,24,3926,3487,939,608
0,1404.3626.txt,35,6247,4804,1072,676
0,1405.0149.txt,34,7257,5066,928,619
0,1409.2612.txt,21,4211,2924,503,309
0,1511.00867.txt,72,14471,12191,1440,921


In [15]:
len(vocabulary)

101460

## Construcción del Bag of Words

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import vstack,save_npz,load_npz

Construido el vocabulario podemos construir el bag of words, que se hace con la ayuda de la funcion CountVectorizer

In [17]:
vectorizer = CountVectorizer(analyzer = "word",vocabulary =vocabulary , tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 5000) 
train_data_features = vectorizer.fit_transform(documents)

In [18]:
len(vocabulary)

101460

In [19]:
save_npz('sparse_matrix.npz', train_data_features)
#sparse_matrix = load_npz('sparse_matrix.npz')

In [20]:
vectorizer.vocabulary_

{'a': 0,
 'aa': 1,
 'aaa': 2,
 'aaaa': 3,
 'aaaaa': 4,
 'aaaaaa': 5,
 'aaaaaaa': 6,
 'aaaaaaaaaaaaaaaababa': 7,
 'aaaab': 8,
 'aaaababa': 9,
 'aaab': 10,
 'aaabbbbaaccccaaaacbbbcccbb': 11,
 'aaai': 12,
 'aaaiaaaipaperview': 13,
 'aaaibarri': 14,
 'aaaimit': 15,
 'aaak': 16,
 'aab': 17,
 'aaba': 18,
 'aabaabc': 19,
 'aabab': 20,
 'aababa': 21,
 'aababaababb': 22,
 'aabb': 23,
 'aabbb': 24,
 'aabc': 25,
 'aabcdaccaac': 26,
 'aabort': 27,
 'aacccgctcgt': 28,
 'aaccept': 29,
 'aachen': 30,
 'aacn': 31,
 'aaction': 32,
 'aad': 33,
 'aadbk': 34,
 'aadom': 35,
 'aaecc': 36,
 'aaeccdoi': 37,
 'aaem': 38,
 'aaf': 39,
 'aag': 40,
 'aagaard': 41,
 'aagea': 42,
 'aah': 43,
 'aai': 44,
 'aaia': 45,
 'aaihaa': 46,
 'aain': 47,
 'aaini': 48,
 'aaixj': 49,
 'aaj': 50,
 'aajbb': 51,
 'aak': 52,
 'aakv': 53,
 'aalbersberg': 54,
 'aalborg': 55,
 'aalcain': 56,
 'aalg': 57,
 'aall': 58,
 'aalter': 59,
 'aaltern': 60,
 'aalto': 61,
 'aam': 62,
 'aama': 63,
 'aamarkov': 64,
 'aamodt': 65,
 'aan': 66,
 'aana

In [21]:
with open('vocabulary.txt', 'w') as f:
    for item in vectorizer.get_feature_names():
        f.write("%s\n" % item)

In [22]:
import pickle
filename = 'BoW1.sav'
pickle.dump(vectorizer, open(filename, 'wb'))
#loaded_model = pickle.load(open('BoW1.sav', 'rb'))

In [23]:
#words = set(nltk.corpus.words.words())

In [24]:
#len(vocabulary)

In [25]:
#print(vocabulary - words)